###  Домашнее задание №1 студентки группы 181 Кошкиной Ксении

In [163]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import RAKE
import nltk
from nltk.corpus import stopwords
import string
from summa import keywords
import yake

#### Тексты взяты с сайта vesti.ru, где внизу каждой статьи прилагаются ключевые слова.

In [164]:
main_df = pd.DataFrame()

In [165]:
def get_text(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
    }
    resp = requests.get(url, headers=headers)
    html = resp.text
    return html

In [166]:
main_df['source'] = ['https://www.vesti.ru/article/2635384', 'https://www.vesti.ru/hitech/article/2635408', \
                    'https://www.vesti.ru/article/2635790', 'https://www.vesti.ru/article/2635756', \
                    'https://www.vesti.ru/article/2635786', 'https://www.vesti.ru/article/2635754', \
                    'https://www.vesti.ru/article/2635941', 'https://www.vesti.ru/article/2636008', \
                     'https://www.vesti.ru/article/2635988']

In [167]:
main_df['text'] = main_df['source'].apply(get_text)

#### Выделеннные мной ключевые слова:

In [168]:
my_keywords1 = ['Милош Земан', 'президент', 'реанимация', 'состояние здоровья', 'правительство', 'Чехия', 'военный госпиталь', 'президентские полномочия']
my_keywords2 = ['деньги', 'мошенники', 'атака', 'жертва мошенников', 'сообщение', 'банковская карта', 'код доступа']
my_keywords3 = ['Аэрофлот', 'Пхукет', 'Таиланд', 'регулярное авиасообщение', 'привитые россияне', 'туристы', 'Бангкок', 'Паттайя']
my_keywords4 = ['Глазго', 'президент', 'Байден', 'климатическая конференция', 'глобальное потепление','углексислый газ', 'изменение климата']
my_keywords5 = ['Томас Бенфилд', 'вакцинация', 'коронавирус', 'симптомы', 'снижение риска заболевания', 'течение болезни']
my_keywords6 = ['Еврейская автономная область', 'прививка', 'вакцинация', 'увольнение', 'работники скорой помощи', 'уходящие медики', 'заявление на увольнение', 'Облучье']
my_keywords7 = ['Новые Ватутинки', 'инцидент', 'следствие', 'драка', 'Александр Жиловников', 'Никита Уткин', 'полиция', 'злоумышленники']
my_keywords8 = ['официальное место работы', 'полиция', 'нападение', 'Новая Москва', 'уроженцы Кавказа', 'Новые Ватутинки', 'маленький сын', 'нож']
my_keywords9 = ['Джон Кеннеди', 'Ли Харви Освальд', 'конспирологи', 'теория заговора', 'убийство Кеннеди', 'спасение Америки', 'торговля детьми', 'Дональд Трамп', 'появление Кеннеди']

In [169]:
main_df['my_keywords'] = [my_keywords1, my_keywords2, my_keywords3, my_keywords4, my_keywords5, \
                         my_keywords6, my_keywords7, my_keywords8, my_keywords9]

#### Функция для того, чтобы достать первичные ключевые слова

In [170]:
def get_keywords(text):
    soup = BeautifulSoup(text,'html.parser')
    finalData = []
    for i in soup.find_all('div', {'class' :'tags'}):
        for element in str(i).split('>'):
            f = re.findall(r'[А-Я]*[а-я]*', element)
            keywords = list(filter(None, f))
            if len(keywords) != 0:
                finalData.append(' '.join(keywords))
    return finalData

In [171]:
main_df['keywords_from_source'] = main_df['text'].apply(get_keywords)

#### Извлекаем из html кода чистый текст:

In [174]:
def get_clean_text(text):
    data = []
    soup = BeautifulSoup(text,'html.parser')
    for i in soup.find_all('div', {'class' :'article__text'}):
        f = re.findall(r'[А-Я]*[а-я]*', str(i))
        for item in f:
            if len(item) != 0:
                data.append(item)
    return ' '.join(data)

In [175]:
main_df['clean_text'] = main_df['text'].apply(get_clean_text)

In [176]:
main_df = main_df.drop(('text'),axis=1)

#### Функция для токенизации текста, взятая из тетрадки семинара:

In [177]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

## Rake

In [178]:
list_stopwords = stopwords.words('russian')

In [179]:
rake = RAKE.Rake(list_stopwords)

In [180]:
main_df['normalize_text'] = main_df['clean_text'].apply(normalize_text)

In [181]:
main_df['rake'] = main_df['normalize_text'].apply(lambda x:
                                                  rake.run(x, maxWords=3, minFrequency=1))

## TextRank

In [182]:
main_df['textrank'] = main_df['normalize_text'].apply(lambda x:
                                                     keywords.keywords(x, language='russian', additional_stopwords=list_stopwords, scores=True))

## Yake

In [183]:
kw_extractor = yake.KeywordExtractor()
max_ngram_size = 2
deduplication_threshold = 0.9
numOfKeywords = 10
custom_kw_extractor = yake.KeywordExtractor(lan='russian', n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)

In [184]:
main_df['yake'] = main_df['normalize_text'].apply(lambda x: custom_kw_extractor.extract_keywords(x))

#### Объединим ключевые слова из статьи и ключевые слова, размеченные мной, и удалим какие-то общие тэги

In [186]:
def concatenation(my_keywords, keywords_from_source):
    list_of_keywords = []
    for i in range(len(my_keywords)):
        final_words = list(set(my_keywords[i] + keywords_from_source[i]))
        if 'общество' in final_words:
            final_words.remove('общество')
        if 'в мире' in final_words:
            final_words.remove('в мире')
        if 'новости' in final_words:
            final_words.remove('новости')
        else:
            pass
        list_of_keywords.append(final_words)
    return list_of_keywords

In [187]:
main_df['combined_keywords'] = concatenation(list(main_df.my_keywords), list(main_df.keywords_from_source))

In [189]:
main_df.head()

,source,my_keywords,keywords_from_source,clean_text,normalize_text,rake,textrank,yake,combined_keywords
0,https://www.vesti.ru/article/2635384,"[Милош Земан, президент, реанимация, состояние...","[общество, в мире, политика, президент, Чехия,...",Медицинский консилиум оценит в пятницу состоян...,медицинский консилиум оценить в пятница состоя...,"[(медицинский консилиум оценить, 9.0), (состоя...","[(земан, 0.1923731054306261), (состояние здоро...","[(состояние здоровье, 0.0005116003525649392), ...","[правительство, состояние здоровья, президент,..."
1,https://www.vesti.ru/hitech/article/2635408,"[деньги, мошенники, атака, жертва мошенников, ...","[общество, происшествия, технологии, обман, об...",Нередко жертвами мошенников становятся те кто ...,нередко жертва мошенник становиться тот кто ра...,"[(якобы проверить действительно, 9.0), (сайт м...","[(атака, 0.21545317808030248), (жертва мошенни...","[(виктор чебышев, 0.002687649831204984), (касп...","[мошенник, обман, технологии, жертва мошеннико..."
2,https://www.vesti.ru/article/2635790,"[Аэрофлот, Пхукет, Таиланд, регулярное авиасоо...","[экономика, Таиланд, Аэрофлот, Бангкок, Пхукет...",Первый за последние месяцев самолет компании А...,один за последний месяц самолёт компания аэроф...,"[(выполнять регулярный авиарейс, 9.0), (прохож...","[(таиланд, 0.22954872965166995), (регулярный, ...","[(составлять миллион, 0.008638577397523413), (...","[Аэрофлот, Паттайя, Пхукет, Таиланд, экономика..."
3,https://www.vesti.ru/article/2635756,"[Глазго, президент, Байден, климатическая конф...","[общество, в мире, встреча, климат, США Америк...",Климатическая конференция которая должна была ...,климатический конференция который должный быть...,"[(главный проблема сша, 9.0), (проблема измене...","[(байден американский, 0.22951372473084455), (...","[(американский президент, 0.000585938267381143...","[глобальное потепление, изменение климата, Бай..."
4,https://www.vesti.ru/article/2635786,"[Томас Бенфилд, вакцинация, коронавирус, симпт...","[общество, медицина, болезнь, Дания, вакцинаци...",Инфекционист Томас Бенфилд и иммунолог Руне Ха...,инфекционист томас бенфилд и иммунолог руно ха...,"[(инфекционист томас бенфилд, 9.0), (это крупн...","[(инфекционист томас бенфилд, 0.48051180797354...","[(томас бенфилд, 0.002684672417102007), (инфек...","[снижение риска заболевания, коронавирус, Дани..."


#### Можно объединить все ключевые слова в один список и сделать морфологические шаблоны через pos-tagging.

In [203]:
def forfor(a):
    return [item for sublist in a for item in sublist]

In [204]:
all_keywords = forfor(list(main_df.combined_keywords))

In [206]:
morph = pymorphy2.MorphAnalyzer()

In [207]:
d = {}
for element in all_keywords:
    sample = []
    element2 = element.split()
    for el in element2:
        pos_element = morph.parse(el)
        sample.append(pos_element[0].tag.POS)
    d[element] = sample

In [208]:
unique_elements = []
for k in list(d.values()):
    if k not in unique_elements:
        unique_elements.append(k)

#### Мы получили список шаблонов на основании части речи:

In [209]:
unique_elements

[['NOUN'],
 ['NOUN', 'NOUN'],
 ['ADJF', 'NOUN'],
 ['PRTF', 'NOUN'],
 ['NOUN', 'NOUN', 'NOUN'],
 ['ADJF', 'ADJF', 'NOUN'],
 ['NOUN', 'PREP', 'NOUN'],
 ['ADJF', 'NOUN', 'NOUN'],
 ['PRCL', 'NOUN', 'NOUN']]

#### Теперь напишем функцию, которая будет размечать ключевые слова, полученные с помощью Rake, TextRank и Yake:

In [57]:
def get_pos_keywords(keywords):
    d = {}
    for el in keywords:
        element = list(el)[0]
        sample = []
        for el2 in element.split():
            pos_element = morph.parse(el2)
            sample.append(pos_element[0].tag.POS)
        d[element] = sample
    return d

#### Воспользуемся функцией, которая будет фильтровать ключевые слова в зависимости от шаблонов:

In [58]:
def sorted_keywords(keywords_dict, unique_elements):
    keywords_sorted = []
    for k, v in keywords_dict.items():
        if v in unique_elements:
            keywords_sorted.append(k)
    return keywords_sorted

In [213]:
main_df['sorted_pos_rake'] = main_df['rake'].apply(lambda x: sorted_keywords(get_pos_keywords(x), unique_elements))
main_df['sorted_pos_textrank'] = main_df['textrank'].apply(lambda x: sorted_keywords(get_pos_keywords(x), unique_elements))
main_df['sorted_pos_yake'] = main_df['yake'].apply(lambda x: sorted_keywords(get_pos_keywords(x), unique_elements))

#### Теперь нужно оценить пересечение с нашими эталонными ключевыми словами. Поскольку некоторые эталонные ключевые слова используются не в начальной форме, то, возможно имеет смысл привести их к начальной форме...

In [62]:
def lemma_keywords(keywords):
    keywords_lemmatized = []
    for el in keywords:
        element = normalize_text(el)
        keywords_lemmatized.append(element)
    return keywords_lemmatized

In [215]:
main_df['lemmatized_keywords'] = main_df['combined_keywords'].apply(lambda x:lemma_keywords(x))

#### Функция для подсчета метрик

In [219]:
main_df.head()

,source,my_keywords,keywords_from_source,clean_text,normalize_text,rake,textrank,yake,combined_keywords,sorted_pos_rake,sorted_pos_textrank,sorted_pos_yake,lemmatized_keywords
0,https://www.vesti.ru/article/2635384,"[Милош Земан, президент, реанимация, состояние...","[общество, в мире, политика, президент, Чехия,...",Медицинский консилиум оценит в пятницу состоян...,медицинский консилиум оценить в пятница состоя...,"[(медицинский консилиум оценить, 9.0), (состоя...","[(земан, 0.1923731054306261), (состояние здоро...","[(состояние здоровье, 0.0005116003525649392), ...","[правительство, состояние здоровья, президент,...","[пониженный артериальный давление, улучшение с...","[земан, состояние здоровье президент, милош зе...","[состояние здоровье, военный госпиталь, госпит...","[правительство, состояние здоровье, президент,..."
1,https://www.vesti.ru/hitech/article/2635408,"[деньги, мошенники, атака, жертва мошенников, ...","[общество, происшествия, технологии, обман, об...",Нередко жертвами мошенников становятся те кто ...,нередко жертва мошенник становиться тот кто ра...,"[(якобы проверить действительно, 9.0), (сайт м...","[(атака, 0.21545317808030248), (жертва мошенни...","[(виктор чебышев, 0.002687649831204984), (касп...","[мошенник, обман, технологии, жертва мошеннико...","[слово виктор чебышев, интернет объявление, бе...","[атака, жертва мошенник, сообщение, этап, мочь...","[виктор чебышев, касперский виктор, чебышев зл...","[мошенник, обман, технология, жертва мошенник,..."
2,https://www.vesti.ru/article/2635790,"[Аэрофлот, Пхукет, Таиланд, регулярное авиасоо...","[экономика, Таиланд, Аэрофлот, Бангкок, Пхукет...",Первый за последние месяцев самолет компании А...,один за последний месяц самолёт компания аэроф...,"[(выполнять регулярный авиарейс, 9.0), (прохож...","[(таиланд, 0.22954872965166995), (регулярный, ...","[(составлять миллион, 0.008638577397523413), (...","[Аэрофлот, Паттайя, Пхукет, Таиланд, экономика...","[страна январь год, обязательный нахождение, п...","[таиланд, пхукет, аэрофлот, самолёт, миллион ч...","[условие бангкок, последний месяц, водяной арка]","[аэрофлот, паттайя, пхукет, таиланд, экономика..."
3,https://www.vesti.ru/article/2635756,"[Глазго, президент, Байден, климатическая конф...","[общество, в мире, встреча, климат, США Америк...",Климатическая конференция которая должна была ...,климатический конференция который должный быть...,"[(главный проблема сша, 9.0), (проблема измене...","[(байден американский, 0.22951372473084455), (...","[(американский президент, 0.000585938267381143...","[глобальное потепление, изменение климата, Бай...","[главный проблема сша, проблема изменение клим...","[президент сша, глазго, администрация, весь ст...","[американский президент, изменение климат, угл...","[глобальный потепление, изменение климат, байд..."
4,https://www.vesti.ru/article/2635786,"[Томас Бенфилд, вакцинация, коронавирус, симпт...","[общество, медицина, болезнь, Дания, вакцинаци...",Инфекционист Томас Бенфилд и иммунолог Руне Ха...,инфекционист томас бенфилд и иммунолог руно ха...,"[(инфекционист томас бенфилд, 9.0), (это крупн...","[(инфекционист томас бенфилд, 0.48051180797354...","[(томас бенфилд, 0.002684672417102007), (инфек...","[снижение риска заболевания, коронавирус, Дани...","[инфекционист томас бенфилд, слово томас бенфи...","[инфекционист томас бенфилд, слово, чаща]","[томас бенфилд, инфекционист томас, страна евр...","[снижение риска заболевание, коронавирус, дани..."


In [239]:
def get_intersection(main_keywords, other_keywords):
    all_keywords = []
    for el in other_keywords:
        if el in main_keywords:
            all_keywords.append(el)
    precision = len(all_keywords) / len(other_keywords)
    recall = len(all_keywords) / len(main_keywords)
    if precision == 0 and recall == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f_score

In [252]:
def get_all_metrics(list1, list2):
    metrics_precision = []
    metrics_recall = []
    metrics_fscore = []
    for i in range(len(list1)):
        metric = get_intersection(list1[i], list2[i])
        metrics_precision.append(metric[0])
        metrics_recall.append(metric[1])
        metrics_fscore.append(metric[2])
    return metrics_precision, metrics_recall, metrics_fscore

### Посчитаем метрики для каждого текста с учетом шаблонов:

#### RAKE

In [255]:
main_df['rake_precision'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_rake))[0]
main_df['rake_recall'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_rake))[1]
main_df['rake_fscore'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_rake))[2]

#### TextRank

In [256]:
main_df['textrank_precision'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_textrank))[0]
main_df['textrank_recall'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_textrank))[1]
main_df['textrank_fscore'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_textrank))[2]

#### YAKE

In [257]:
main_df['yake_precision'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_yake))[0]
main_df['yake_recall'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_yake))[1]
main_df['yake_fscore'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.sorted_pos_yake))[2]

In [284]:
main_df

,source,my_keywords,keywords_from_source,clean_text,normalize_text,rake,textrank,yake,combined_keywords,sorted_pos_rake,...,clean_keywords_yake,rake_precision_without_patterns,rake_recall_without_patterns,rake_fscore_without_patterns,textrank_precision_without_patterns,textrank_recall_without_patterns,textrank_fscore_without_patterns,yake_precision_without_patterns,yake_recall_without_patterns,yake_fscore_without_patterns
0,https://www.vesti.ru/article/2635384,"[Милош Земан, президент, реанимация, состояние...","[общество, в мире, политика, президент, Чехия,...",Медицинский консилиум оценит в пятницу состоян...,медицинский консилиум оценить в пятница состоя...,"[(медицинский консилиум оценить, 9.0), (состоя...","[(земан, 0.1923731054306261), (состояние здоро...","[(состояние здоровье, 0.0005116003525649392), ...","[правительство, состояние здоровья, президент,...","[пониженный артериальный давление, улучшение с...",...,"[состояние здоровье, военный госпиталь, госпит...",0.046875,0.300000,0.081081,0.054054,0.200000,0.085106,0.2,0.200000,0.200000
1,https://www.vesti.ru/hitech/article/2635408,"[деньги, мошенники, атака, жертва мошенников, ...","[общество, происшествия, технологии, обман, об...",Нередко жертвами мошенников становятся те кто ...,нередко жертва мошенник становиться тот кто ра...,"[(якобы проверить действительно, 9.0), (сайт м...","[(атака, 0.21545317808030248), (жертва мошенни...","[(виктор чебышев, 0.002687649831204984), (касп...","[мошенник, обман, технологии, жертва мошеннико...","[слово виктор чебышев, интернет объявление, бе...",...,"[виктор чебышев, касперский виктор, чебышев зл...",0.071429,0.153846,0.097561,0.187500,0.230769,0.206897,0.0,0.000000,0.000000
2,https://www.vesti.ru/article/2635790,"[Аэрофлот, Пхукет, Таиланд, регулярное авиасоо...","[экономика, Таиланд, Аэрофлот, Бангкок, Пхукет...",Первый за последние месяцев самолет компании А...,один за последний месяц самолёт компания аэроф...,"[(выполнять регулярный авиарейс, 9.0), (прохож...","[(таиланд, 0.22954872965166995), (регулярный, ...","[(составлять миллион, 0.008638577397523413), (...","[Аэрофлот, Паттайя, Пхукет, Таиланд, экономика...","[страна январь год, обязательный нахождение, п...",...,"[составлять миллион, превысить миллион, вылечи...",0.105263,0.200000,0.137931,0.200000,0.300000,0.240000,0.0,0.000000,0.000000
3,https://www.vesti.ru/article/2635756,"[Глазго, президент, Байден, климатическая конф...","[общество, в мире, встреча, климат, США Америк...",Климатическая конференция которая должна была ...,климатический конференция который должный быть...,"[(главный проблема сша, 9.0), (проблема измене...","[(байден американский, 0.22951372473084455), (...","[(американский президент, 0.000585938267381143...","[глобальное потепление, изменение климата, Бай...","[главный проблема сша, проблема изменение клим...",...,"[американский президент, изменение климат, угл...",0.042857,0.272727,0.074074,0.030303,0.090909,0.045455,0.1,0.090909,0.095238
4,https://www.vesti.ru/article/2635786,"[Томас Бенфилд, вакцинация, коронавирус, симпт...","[общество, медицина, болезнь, Дания, вакцинаци...",Инфекционист Томас Бенфилд и иммунолог Руне Ха...,инфекционист томас бенфилд и иммунолог руно ха...,"[(инфекционист томас бенфилд, 9.0), (это крупн...","[(инфекционист томас бенфилд, 0.48051180797354...","[(томас бенфилд, 0.002684672417102007), (инфек...","[снижение риска заболевания, коронавирус, Дани...","[инфекционист томас бенфилд, слово томас бенфи...",...,"[томас бенфилд, инфекционист томас, страна евр...",0.045455,0.111111,0.064516,0.000000,0.000000,0.000000,0.1,0.111111,0.105263
5,https://www.vesti.ru/article/2635754,"[Еврейская автономная область, прививка, вакци...","[общество, медицина, скорая помощь, увольнение...",В Еврейской автономной области работники скоро...,в еврейский автономный область работник скора ...,"[(семейный обстоятельство однако, 9.0), (начат...","[(сотрудник, 0.18292162884866434), (работник с...","[(скора помощь, 0.0012857469940610112), (раб

### Посчитаем метрики без учета шаблонов:

#### Функция для извлечения ключевых слов без значений:

In [68]:
def clean_keywords(keywords):
    keywordsClean = []
    for el in keywords:
        keywordsClean.append(el[0])
    return keywordsClean

In [261]:
main_df['clean_keywords_rake'] = main_df['rake'].apply(lambda x: clean_keywords(x))

In [262]:
main_df['clean_keywords_textrank'] = main_df['textrank'].apply(lambda x: clean_keywords(x))

In [263]:
main_df['clean_keywords_yake'] = main_df['yake'].apply(lambda x: clean_keywords(x))

#### RAKE

In [268]:
main_df['rake_precision_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_rake))[0]
main_df['rake_recall_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_rake))[1]
main_df['rake_fscore_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_rake))[2]



#### TextRank

In [269]:
main_df['textrank_precision_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_textrank))[0]
main_df['textrank_recall_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_textrank))[1]
main_df['textrank_fscore_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_textrank))[2]

#### YAKE

In [270]:
main_df['yake_precision_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_yake))[0]
main_df['yake_recall_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_yake))[1]
main_df['yake_fscore_without_patterns'] = get_all_metrics(list(main_df.lemmatized_keywords), list(main_df.clean_keywords_yake))[2]

In [272]:
rake_scores = main_df[['rake_precision', 'rake_recall', 'rake_fscore', 'rake_precision_without_patterns', 'rake_recall_without_patterns', 'rake_fscore_without_patterns']]

In [279]:
rake_scores

,rake_precision,rake_recall,rake_fscore,rake_precision_without_patterns,rake_recall_without_patterns,rake_fscore_without_patterns
0,0.090909,0.300000,0.139535,0.046875,0.300000,0.081081
1,0.153846,0.153846,0.153846,0.071429,0.153846,0.097561
2,0.153846,0.200000,0.173913,0.105263,0.200000,0.137931
3,0.071429,0.272727,0.113208,0.042857,0.272727,0.074074
4,0.071429,0.111111,0.086957,0.045455,0.111111,0.064516
5,0.200000,0.181818,0.190476,0.060606,0.181818,0.090909
6,0.029412,0.100000,0.045455,0.014493,0.100000,0.025316
7,0.066667,0.100000,0.080000,0.025641,0.100000,0.040816
8,0.078947,0.230769,0.117647,0.034884,0.230769,0.060606


In [278]:
rake_scores.describe()

,rake_precision,rake_recall,rake_fscore,rake_precision_without_patterns,rake_recall_without_patterns,rake_fscore_without_patterns
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.101832,0.183364,0.122337,0.049722,0.183364,0.074757
std,0.054796,0.074252,0.046974,0.026900,0.074252,0.033003
min,0.029412,0.100000,0.045455,0.014493,0.100000,0.025316
25%,0.071429,0.111111,0.086957,0.034884,0.111111,0.060606
50%,0.078947,0.181818,0.117647,0.045455,0.181818,0.074074
75%,0.153846,0.230769,0.153846,0.060606,0.230769,0.090909
max,0.200000,0.300000,0.190476,0.105263,0.300000,0.137931


In [274]:
textrank_scores = main_df[['textrank_precision', 'textrank_recall', 'textrank_fscore', 'textrank_precision_without_patterns', 'textrank_recall_without_patterns', 'textrank_fscore_without_patterns']]

In [275]:
textrank_scores

,textrank_precision,textrank_recall,textrank_fscore,textrank_precision_without_patterns,textrank_recall_without_patterns,textrank_fscore_without_patterns
0,0.076923,0.200000,0.111111,0.054054,0.200000,0.085106
1,0.272727,0.230769,0.250000,0.187500,0.230769,0.206897
2,0.272727,0.300000,0.285714,0.200000,0.300000,0.240000
3,0.047619,0.090909,0.062500,0.030303,0.090909,0.045455
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.250000,0.181818,0.210526,0.125000,0.181818,0.148148
6,0.117647,0.200000,0.148148,0.093750,0.300000,0.142857
7,0.200000,0.100000,0.133333,0.050000,0.100000,0.066667
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [280]:
textrank_scores.describe()

,textrank_precision,textrank_recall,textrank_fscore,textrank_precision_without_patterns,textrank_recall_without_patterns,textrank_fscore_without_patterns
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.137516,0.144833,0.133481,0.082290,0.155944,0.103903
std,0.113518,0.103554,0.102516,0.074972,0.114951,0.086134
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.047619,0.090909,0.062500,0.030303,0.090909,0.045455
50%,0.117647,0.181818,0.133333,0.054054,0.181818,0.085106
75%,0.250000,0.200000,0.210526,0.125000,0.230769,0.148148
max,0.272727,0.300000,0.285714,0.200000,0.300000,0.240000


In [281]:
yake_scores = main_df[['yake_precision', 'yake_recall', 'yake_fscore', 'yake_precision_without_patterns', 'yake_recall_without_patterns', 'yake_fscore_without_patterns']]

In [282]:
yake_scores

,yake_precision,yake_recall,yake_fscore,yake_precision_without_patterns,yake_recall_without_patterns,yake_fscore_without_patterns
0,0.200000,0.200000,0.200000,0.2,0.200000,0.200000
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,0.125000,0.090909,0.105263,0.1,0.090909,0.095238
4,0.166667,0.111111,0.133333,0.1,0.111111,0.105263
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,0.333333,0.100000,0.153846,0.2,0.200000,0.200000
7,0.333333,0.100000,0.153846,0.1,0.100000,0.100000
8,0.285714,0.153846,0.200000,0.2,0.153846,0.173913


In [283]:
yake_scores.describe()

,yake_precision,yake_recall,yake_fscore,yake_precision_without_patterns,yake_recall_without_patterns,yake_fscore_without_patterns
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.160450,0.083985,0.105143,0.100000,0.095096,0.097157
std,0.139446,0.071412,0.084187,0.086603,0.081309,0.083247
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.166667,0.100000,0.133333,0.100000,0.100000,0.100000
75%,0.285714,0.111111,0.153846,0.200000,0.153846,0.173913
max,0.333333,0.200000,0.200000,0.200000,0.200000,0.200000


### Главные выводы и ошибки: 
1. ключевые слова разных методов пересекаются редко.
2. выделяются глаголы, которые в русском языке не используются как ключевые слова. Возможно, решением данной проблемы будет являться перевод из одной части речи в другую, то есть из глагола в существительное.
3. ключевые слова методов и эталонные ключевые слова не пересекаются из несоответствия длины данных ключевых слов. Например, в эталоне выделяется "Томас Бенфилд", а в ключевых словах, полученных с помощью методов содержит почти такое же, но "инфекционист Томас Бенфилд". Из-за неполного соответствия данный пример уже не включается в пересечение между эталонными ключевыми словами и ключевыми словами, полученными с помощью методов. Эту проблему можно решить: проверять ключевые слова не на полное соответствие, а на включение одного множества в другое (как в данном примере) и не выкидывать этот пример, когда делаем подсчета метрик.
4. Метрики получились не самыми лучшими.